## Functions

__Function for getting the dynamic parameters of the cell model__

In [2]:
def get_dyn_param(data,cell_temp):    
    
    ind = np.where(np.array(data['temps']) == cell_temp)[0][0]

    eta = data['etaParam'][ind]
    Q = data['QParam'][ind]
    gamma = data['GParam'][ind]
    M0 = data['M0Param'][ind]
    M = data['MParam'][ind]
    R0 = data['R0Param'][ind]
    RC = data['RCParam'][ind]
    R = data['RParam'][ind]

    return (eta,Q,gamma,M0,M,R0,RC,R)

In [74]:
from scipy.interpolate import interp1d

def interp_ocv(z,OCV):
    
    dum = interp1d(z,OCV)
    
    return dum

__OCV calculation for any SOC value Function__

In [81]:
def OCV_from_SOC(z,temp,SOC,OCV_0,OCV_rel):

    # OCV calculation from the given SOC (Coloumb Counting)

    OCV_0_z = interp_ocv(SOC,OCV_0)(z)
    OCV_rel_z = interp_ocv(SOC,OCV_rel)(z)

    ocv_z = OCV_0_z + temp*OCV_rel_z

    return (ocv_z)

## Main Code

__Importing the Dynamic model__

In [3]:
import json
  
# Opening JSON file
f = open('modeldyn.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

__User Inputs__

In [89]:
# Number of cells in series in a module
Ns = 5

# Number of modules in parallel
Np = 3

# Simulation Time
max_time = 10

# Pack rests at time t0
t0 = 5

# Storage variable
z_pack = np.zeros((max_time,Ns,Np))
i_pack = np.zeros((max_time,Ns,Np))

# Initial state of the cells in the pack
z_ini = 0.25*np.ones((Ns,Np))
irc = np.zeros((Ns,Np))
h = np.zeros((Ns,Np))

# Cell Interconnect resistance
R_inter = 125e-6

# Cell Temperature
# It is assumed that the cell temperature remains constant at 25°C
cell_temp = 25

# Random values initialization
random_init = 1

In [53]:
# Cell Parametesr calculated from dynamic model

eta = get_dyn_param(data,cell_temp)[0]*np.ones((Ns,Np)) 
Q = get_dyn_param(data,cell_temp)[1]*np.ones((Ns,Np)) 
gamma = get_dyn_param(data,cell_temp)[2]*np.ones((Ns,Np)) 
M0 = get_dyn_param(data,cell_temp)[3]*np.ones((Ns,Np)) 
M = get_dyn_param(data,cell_temp)[4]*np.ones((Ns,Np)) 
R0 = get_dyn_param(data,cell_temp)[5]*np.ones((Ns,Np)) 
RC = get_dyn_param(data,cell_temp)[6]*np.ones((Ns,Np)) 
R = get_dyn_param(data,cell_temp)[7]*np.ones((Ns,Np)) 

In [60]:
# Fault parameters

#1.Open Cicuit fault
# Cell will have infinite resistance
R0_open = np.inf

#2.Short circuit fault
# During short circuit SOC cannot be measured
# We will assume some resistance for the cell

z_short = np.nan

R0_short = 0.025

In [70]:
if random_init:
    
    z = 0.25 + 0.4*np.random.rand(Ns,Np)
    Q = 4.5 + np.random.rand(Ns,Np)
    R0 = 0.005 + 0.020*np.random.rand(Ns,Np)

In [71]:
# Updating the cell resistance with interconnect resistance

R0 = R0 + 2*R_inter

__Simulation Parameters__

In [108]:
# Pack capcity is minimum module capacity

module_cap = np.sum(Q, axis = 1)
pack_cap = min(module_cap)

# Charge/Discharge current
# 10C charge/discharge

I = pack_cap*10

# Introducing the faulty cell
z[3][2] = z_short

In [77]:
SOC = data['SOC']
OCV_0  = data['OCV0']
OCV_rel = data['OCVrel']

In [121]:
# Terminal Voltage Calculation

v = OCV_from_SOC(z,cell_temp,SOC,OCV_0,OCV_rel)
v = v + np.multiply(M,h) - np.multiply(R,irc)

# Setting the short-circuit cell voltage to zero
v[np.isnan(v)] = 0

In [123]:
# If the cell is short-ciruit then current is going to external resitor
R0[np.isnan(z)] = R0_short

In [191]:
# Pack voltage calculation

# Module current calculation
im = (np.sum(np.divide(v,R0),axis = 1) - I)

# Module equivalent resistance reciprocal
r_invm = np.sum(np.divide(1,R0),axis = 1)

# Module voltage
Vm = np.divide(im,r_invm)

# Pack Voltage
V_pack = sum(Vm)

# Cell voltage calculation
v - np.tile(Vm,3).reshape(3,5).T

# Cell current calculation
i_cell = np.divide(v,R0)

# Cell new SOC calculation
z = z-(1/3600)*(np.divide(i,Q))

# Short-circuiting the cell below discharge capacity
z[z<0] = np.nan

# Updating the capacitor volage
irc = np.multiply(RC,irc) + np.multiply((1-RC),i_cell)

In [192]:
irc

array([[ 6.11909844e+01,  1.88439581e+00, -2.82488567e+03],
       [ 2.04065115e+01,  1.21785545e+00, -9.36344921e+03],
       [ 1.96700168e+01,  1.10797258e+00, -3.13584045e+03],
       [ 1.72317258e+01,  1.46799212e+00,  0.00000000e+00],
       [ 5.38326786e+01,  1.09342610e+00, -6.07147413e+03]])

In [193]:
i_cell

array([[533.07174638, 259.51936081, 138.49147128],
       [177.77348796, 167.72329154, 459.04790783],
       [171.3574362 , 152.59020193, 153.73618914],
       [150.11600552, 202.1721635 ,   0.        ],
       [468.9690854 , 150.58685779, 297.6571385 ]])

In [194]:
I

145.87293917911842